In [1]:
import numpy as np
from cookiecutter.main import cookiecutter
import collections
from pathlib import Path
import json
import shutil

In [2]:
SEED = 178 
WORLD_NAME = 'segmentation_world' #NAME OF SDF OUTPUT FILA
MODEL_NAME_PREFIX = 'tomato' #NAME OF MODEL IN SDF FILE
OUT_PATH = Path(Path.cwd() / '../generated').resolve() #PATH OF GENERATE MODELS AND SDF FILE    
MODEL_TEMPLATE = Path(Path.cwd() / '../templates/tomato_model').resolve() #PATH OF BLENDER MODEL, DONT CHANGE
WORLD_TEMPLATE = Path(Path.cwd() / '../templates/tomato_world').resolve() #PATH OF SDF TEMPLANTE, DONT CHANGE
ROW_COUNT = 1 #NUMBER OF ROWS
ROW_LENGTH = 1 #NUMBER OF PLANTS PER ROW


shutil.rmtree(OUT_PATH, ignore_errors=True)
np.random.seed(SEED)


In [3]:
# helper class to build the markers.json
class Markers:
    markers = []
    last_id = 0

    @staticmethod
    def next_id():
        Markers.last_id += 1
        return Markers.last_id

    @staticmethod
    def reset():
        Markers.markers = []
    
    @staticmethod
    def add_plant(x, y, z):
        id = Markers.next_id()
        Markers.markers.append({
            'marker_type': 'PLANT',
            'id': id,
            'translation': [x, y, z]
        })
        return id
    

    @staticmethod
    def add_fruit(x, y, z, plant_id):
        id = Markers.next_id()
        Markers.markers.append({
            'marker_type': 'FRUIT',
            'id': id,
            'translation': [x, y, z],
            'plant_id': plant_id
        })
        return id
    
    @staticmethod
    def dumps():
        return json.dumps(Markers.markers, indent=4)

In [4]:
models = {'list': []}
Markers.reset()
#LOOP FOR GENERATE THE MODEL AND LINK IT TO THE SDF FILE
for x in range(ROW_COUNT):
    for y in range(ROW_LENGTH):
        model_name = f'{MODEL_NAME_PREFIX}_{x * ROW_LENGTH + y}'

        cookiecutter(str(MODEL_TEMPLATE),
             output_dir=str(OUT_PATH), 
             overwrite_if_exists=True, 
             no_input=True,
             extra_context={'world_name': WORLD_NAME, 'model_name': model_name})
        
        x_pos, y_pos, z_pos = 0, 0, 0  # Position at (0, 0, 0)

        
        models['list'].append({
            'model': model_name,
            'name': model_name,
            'pose': f'{x_pos} {y_pos} {z_pos} 0 0 0' 
        })
        
        seed = np.random.randint(10000)
        dir = (OUT_PATH / WORLD_NAME / model_name).resolve()
        dir_blender = (Path.cwd() / '../blender').resolve()
        blend = str(dir_blender / 'tomato_gen.blend')
        script = str(dir_blender / 'tomato_gen.py')
        ! blender $blend --background --python $script -- --model_dir $dir --seed $seed

        plant_id = Markers.add_plant(x_pos, y_pos, z_pos)
        with open(dir / 'markers.json') as markers_file:
            plant_markers = json.load(markers_file)
            for marker in plant_markers:
                if marker['marker_type'] == 'FRUIT':
                    Markers.add_fruit(
                        marker['translation'][0] + x_pos,
                        marker['translation'][1] + y_pos,
                        marker['translation'][2] + z_pos,
                        plant_id
                    )
            

cookiecutter(str(WORLD_TEMPLATE),
             output_dir=str(OUT_PATH), 
             overwrite_if_exists=True, 
             no_input=True,
             extra_context={'world_name': WORLD_NAME, 'models': models})

with open(OUT_PATH / WORLD_NAME / 'markers.json', 'w') as outfile:
    json.dump(Markers.markers, outfile, indent=4, sort_keys=True)

Color management: using fallback mode for management
Color management: Error could not find role data role.
Blender 3.0.1
Read prefs: /home/felipe/.config/blender/3.0/config/userpref.blend
Color management: scene view "Filmic" not found, setting default "Standard".
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Checking requirements:
/bin/python3.10: No module named ensurepip
  Checking yaml
  Checking numpy
  Checking scipy
  Checking pkg_resources
  Checking collada
  Checking pydot
  Checking lxml
  Checking networkx
  Checking trimesh
  Checking PIL
Importing phobos
IMPORT:  phobos.blender.defs
Parsing definitions from: /home/felipe/.config/blender/3.0/scripts/addons/phobos/data/blender/definitions
  defaultControllers.yml
  defaultMaterials.yml
  defaultSensors.yml
  defaultMotors.yml
  def